In [2]:
import tensorflow as tf

In [86]:
import cv2
import tensorflow.keras.layers as tl
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split # LSTM을 사용할 것이라 딱히 필요없을 것으로 보임
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, MaxAbsScaler # Scipy에 제시된 대표적인 스케일링 기법
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from tqdm import tqdm
import csv

### 영상 데이터를 프레임별로 쪼개는 작업

In [4]:
WIDTH = 360
HEIGHT = 540

In [63]:
import os
path = os.getcwd()
print("current path:",path)
hoon_video_path = path + "/test3.mp4"

current path: /Users/ichanho/workspace


In [64]:
hoon_file_list = glob(hoon_video_path, recursive=True)
hoon_file_list = hoon_file_list[1:]
hoon_file_list

[]

In [65]:
!mkdir hoon_images

mkdir: hoon_images: File exists


In [ ]:
!rm -rf hoon_images/*


In [70]:
cap = cv2.VideoCapture(hoon_video_path)
frame_count = 0
status, image = cap.read()
image = cv2.flip(image, 0)
image = cv2.flip(image, 1)
while(status):
    cv2.imwrite("./hoon_images/input_%05d.jpg"%frame_count, image)
    status, image = cap.read()
    image = cv2.flip(image, 0)
    frame_count = frame_count +1
cap.release()

In [71]:
!rm -rf converted_image/*

In [72]:
def output_keypoints(frame, proto_file, weights_file, threshold, model_name, BODY_PARTS):
    global points

    # 네트워크 불러오기
    net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)

    # 입력 이미지의 사이즈 정의
    image_height = HEIGHT
    image_width = WIDTH

    # 네트워크에 넣기 위한 전처리
    input_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (image_width, image_height), (0, 0, 0), swapRB=False, crop=False)

    # 전처리된 blob 네트워크에 입력
    net.setInput(input_blob)

    # 결과 받아오기
    out = net.forward()
    # The output is a 4D matrix :
    # The first dimension being the image ID ( in case you pass more than one image to the network ).
    # The second dimension indicates the index of a keypoint.
    # The model produces Confidence Maps and Part Affinity maps which are all concatenated.
    # For COCO model it consists of 57 parts – 18 keypoint confidence Maps + 1 background + 19*2 Part Affinity Maps. Similarly, for MPI, it produces 44 points.
    # We will be using only the first few points which correspond to Keypoints.
    # The third dimension is the height of the output map.
    out_height = out.shape[2]
    # The fourth dimension is the width of the output map.
    out_width = out.shape[3]

    # 원본 이미지의 높이, 너비를 받아오기
    frame_height, frame_width = frame.shape[:2]

    # 포인트 리스트 초기화
    points = []

    # print(f"\n============================== {model_name} Model ==============================")
    for i in range(len(BODY_PARTS)):
        if((i>=0)and(i<=7)):
            continue
        if(i==25):
            continue
        if((i>=15)and(i<=18)):
            continue
        
        # 신체 부위의 confidence map
        prob_map = out[0, i, :, :]

        # 최소값, 최대값, 최소값 위치, 최대값 위치
        min_val, prob, min_loc, point = cv2.minMaxLoc(prob_map)

        # 원본 이미지에 맞게 포인트 위치 조정
        x = (frame_width * point[0]) / out_width
        x = int(x)
        y = (frame_height * point[1]) / out_height
        y = int(y)

        if prob > threshold:  # [pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, lineType=cv2.LINE_AA)

            points.append((x, y))
            # print(f"[pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

        else:  # [not pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)

            points.append(None)
            # print(f"[not pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

    #cv2_imshow("Output_Keypoints", frame)
    #cv2.imshow("Walking Vision Test", frame)
    #cv2.waitKey(10)
    
    return frame

In [109]:
def make_csv_and_image(frame, proto_file, weights_file, threshold, model_name, BODY_PARTS):
    global points

    # 네트워크 불러오기
    net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)

    # 입력 이미지의 사이즈 정의
    image_height = HEIGHT
    image_width = WIDTH

    # 네트워크에 넣기 위한 전처리
    input_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (image_width, image_height), (0, 0, 0), swapRB=False, crop=False)

    # 전처리된 blob 네트워크에 입력
    net.setInput(input_blob)

    # 결과 받아오기
    out = net.forward()
    out_height = out.shape[2]
    out_width = out.shape[3]

    # 원본 이미지의 높이, 너비를 받아오기
    frame_height, frame_width = frame.shape[:2]

    # 포인트 리스트 초기화
    points = []

    # print(f"\n============================== {model_name} Model ==============================")
    
    ###############################################################################################
    # Making The CSV File
    # 1. make the columns
    # 2. input the data
    # 

    
    for i in range(len(BODY_PARTS)):
        if((i>=0)and(i<=7)):
            continue
        if(i==25):
            continue
        if((i>=15)and(i<=18)):
            continue
        
        # 신체 부위의 confidence map
        prob_map = out[0, i, :, :]

        # 최소값, 최대값, 최소값 위치, 최대값 위치
        min_val, prob, min_loc, point = cv2.minMaxLoc(prob_map)

        # 원본 이미지에 맞게 포인트 위치 조정
        x = (frame_width * point[0]) / out_width
        x = int(x)
        y = (frame_height * point[1]) / out_height
        y = int(y)
        #if(i==10):
        #    print("RKnee:", (x, y))
        #if(i==13):
        #    print("LKnee:", (x, y))
        if((i>=0)and(i<=7)):
            continue
        if(i==25):
            continue
        if((i>=15)and(i<=18)):
            continue

        if prob > threshold:  # [pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, lineType=cv2.LINE_AA)

            points.append(x)
            points.append(y)
            #print(f"[pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

        else:  # [not pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)

            points.append(x)
            points.append(y)
            #points.append(None)
            #print(f"[not pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

    #np_list = np.array(points)
    #cv2_imshow("Output_Keypoints", frame)
    #cv2.imshow("Walking Vision Test", frame)
    #cv2.waitKey(10)
    
    return frame, points

In [110]:
def output_keypoints_with_lines(frame, POSE_PAIRS):
    print()
    for pair in POSE_PAIRS:
        print(pair)
        part_a = pair[0]  # 0 (Head)
        part_b = pair[1]  # 1 (Neck)
        if points[part_a] and points[part_b]:
            print(f"[linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")
            cv2.line(frame, points[part_a], points[part_b], (0, 255, 0), 3)
        else:
            print(f"[not linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")

    #cv2_imshow("output_keypoints_with_lines", frame)
    cv2.imshow("walking vision", frame)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    

In [111]:
BODY_PARTS_BODY_25 = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                      5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "MidHip", 9: "RHip",
                      10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee", 14: "LAnkle",
                      15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe",
                      20: "LSmallToe", 21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel", 25: "Background"}

POSE_PAIRS_BODY_25 = [[0, 1], [0, 15], [0, 16], [1, 2], [1, 5], [1, 8], [8, 9], [8, 12], [9, 10], [12, 13], [2, 3],
                      [3, 4], [5, 6], [6, 7], [10, 11], [13, 14], [15, 17], [16, 18], [14, 21], [19, 21], [20, 21],
                      [11, 24], [22, 24], [23, 24]]

In [113]:
protoFile_body_25 = "./openpose/models/pose/body_25/pose_deploy.prototxt"
weightsFile_body_25 = "./openpose/models/pose/body_25/pose_iter_584000.caffemodel"

# 이미지 경로
!mkdir converted_image
hoon_file_list = glob("./hoon_images/**.jpg", recursive=True)
hoon_file_list.sort()
# print(hoon_file_list[0][14:])
convert_path = "./converted_image/"
file = open("./data.csv", 'w', newline='')
wr = csv.writer(file)
for i in tqdm(hoon_file_list):
    man = i
    name = convert_path + i[14:]
    # 키포인트를 저장할 빈 리스트
    points = []

    # 이미지 읽어오기
    frame_mpii = cv2.imread(man)
    frame_body_25 = frame_mpii.copy()

    frame_BODY_25, info = make_csv_and_image(frame=frame_body_25, proto_file=protoFile_body_25, weights_file=weightsFile_body_25,threshold=0.2, model_name="BODY_25", BODY_PARTS=BODY_PARTS_BODY_25)
    del(info[15:19])
    del(info[0:8])
    del(info[-1])
    cv2.imwrite(name, frame_BODY_25)
    wr.writerow(info)
file.close()
    

mkdir: converted_image: File exists


100%|██████████| 947/947 [23:37<00:00,  1.50s/it]


In [114]:
input_data = open("./data.csv", 'r')
read = csv.reader(input_data)
mydata = []

cnt = 0
for i in read:
    if(cnt == 0):
        cnt += 1
        continue
    if(cnt >= 940):
        cnt += 1
        continue
    
    mydata.append(i)
    cnt += 1

input_data.close()
train = np.array(mydata)
print(train.shape)

(939, 13)


### 프레임 별로 쪼갠 데이터를 Sequence화 하는 작업

In [115]:
seq_length = 60

In [116]:
def Sequence(args):
    full_seq_data = []
    for seq in range(len(args) - seq_length):
        full_seq_data.append(args[seq:seq + seq_length])
    full_seq_data = np.array(full_seq_data)
    return full_seq_data

In [149]:
train_x = Sequence(train)
print(train_x.shape)

(879, 60, 13)


In [150]:
train_y = []
ans = 1

for i in range(train_x.shape[0]):
    train_y.append(ans)
train_y = np.array(train_y)
train_y.shape

(879,)

### 모델 생성 및 컴파일

In [143]:
model = Sequential()

In [144]:
model.add(tl.LSTM(64, return_sequences = True, activation='softmax', input_shape = train_x.shape[1:]))
model.add(tl.Dropout(0.2))
model.add(tl.Dense(100, activation = None))
model.add(tl.Dense(3, activation = 'softmax'))

In [145]:
model.compile(loss = 'mse', optimizer = 'Adam')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 60, 64)            19968     
                                                                 
 dropout_2 (Dropout)         (None, 60, 64)            0         
                                                                 
 dense_4 (Dense)             (None, 60, 100)           6500      
                                                                 
 dense_5 (Dense)             (None, 60, 3)             303       
                                                                 
Total params: 26,771
Trainable params: 26,771
Non-trainable params: 0
_________________________________________________________________


In [146]:
early_stopping = EarlyStopping(patience=5)

In [151]:
model.fit(train_x, train_y, epochs=50, callbacks=early_stopping)

Epoch 1/50


UnimplementedError:  Cast string to float is not supported
	 [[node sequential_2/Cast
 (defined at /opt/anaconda3/envs/openpose/lib/python3.8/site-packages/keras/engine/functional.py:671)
]] [Op:__inference_train_function_6279]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_2/Cast:
In[0] IteratorGetNext (defined at /opt/anaconda3/envs/openpose/lib/python3.8/site-packages/keras/engine/training.py:866)

Operation defined at: (most recent call last)
>>>   File "/opt/anaconda3/envs/openpose/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/opt/anaconda3/envs/openpose/lib/python3.8/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/opt/anaconda3/envs/openpose/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "/opt/anaconda3/envs/openpose/lib/python3.8/si